In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
input_shape = (224, 224, 3)

In [5]:
base_model = MobileNet(include_top=False, input_shape=input_shape, weights = "imagenet")

17225924/17225924 [==============================] - 0s 0us/step


In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(5, activation='softmax')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=x)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train = '/content/drive/MyDrive/Data/train'

In [12]:
val = '/content/drive/MyDrive/Data/val'

In [13]:
# Define the batch size and number of epochs for training
batch_size = 60
epochs = 60

In [14]:
# Create the generators for the training and validation sets
train_generator = train_datagen.flow_from_directory(train,
                                                    target_size=(224,224),
                                                    batch_size=batch_size,
                                                    class_mode="categorical"
                                                    )
validation_generator = test_datagen.flow_from_directory(val,
                                                        target_size=(224,224),
                                                        batch_size=batch_size,
                                                        class_mode="categorical"
                                                        )

Found 2327 images belonging to 5 classes.
Found 520 images belonging to 5 classes.


In [ ]:
model.fit(train_generator,
          epochs=epochs,
          validation_data=validation_generator,steps_per_epoch=30)

Epoch 1/60
 7/30 [======>.......................] - ETA: 5:59 - loss: 0.8041 - accuracy: 0.7310

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


30/30 [==============================] - 625s 20s/step - loss: 0.6765 - accuracy: 0.8019 - val_loss: 5.0920 - val_accuracy: 0.3769
Epoch 2/60
30/30 [==============================] - 464s 15s/step - loss: 0.3163 - accuracy: 0.9032 - val_loss: 2.9218 - val_accuracy: 0.5346
Epoch 3/60
30/30 [==============================] - 480s 16s/step - loss: 0.2132 - accuracy: 0.9278 - val_loss: 5.2019 - val_accuracy: 0.4673
Epoch 4/60
30/30 [==============================] - 472s 16s/step - loss: 0.1173 - accuracy: 0.9608 - val_loss: 1.1843 - val_accuracy: 0.8096
Epoch 5/60
30/30 [==============================] - 471s 16s/step - loss: 0.1462 - accuracy: 0.9541 - val_loss: 2.9373 - val_accuracy: 0.6212
Epoch 6/60
30/30 [==============================] - 478s 16s/step - loss: 0.1253 - accuracy: 0.9536 - val_loss: 3.9076 - val_accuracy: 0.5538
Epoch 7/60
11/30 [==========>...................] - ETA: 4:22 - loss: 0.0745 - accuracy: 0.9784

In [ ]:
model.save("mobilenet.h5")

In [ ]:
model =tf.keras.models.load_model("mobilenet.h5",compile=False)

#Convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] #Uses default optimization strategy to reduce the model size
#converter.target_spec.supported_types = [tf.uint8]
tflite_model = converter.convert()
open("mobile1S.tflite", "wb").write(tflite_model)